# Provider

A collection of Provider implementations that will take the current date and provide an
item valid for that date.

In [13]:
from calendar import TUESDAY, THURSDAY
from datetime import date, timedelta
from typing import List, Tuple

from lib.provider import \
    NeverProvider, \
    AlwaysProvider, \
    ScheduledProvider, \
    AnyProvider, \
    AllProvider
from lib.schedule import \
    WeeklySchedule, \
    AnySchedule, \
    UntilSchedule

START_DATE = date.today()


def print_value_list(values: List[Tuple[date, str]]) -> None:
    print('[', end='')
    print(*[d.strftime(f'%Y-%m-%d : %a : {v}') for d, v in values], end='', sep='\n ')
    print(*values, end='', sep='\n ')
    print(']')

## NeverProvider

This is a trivial provider that always provides `None`

In [14]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
items = zip(days, map(NeverProvider[str]().get, days))
print_value_list(items)

[2025-07-31 : Thu : None
 2025-08-01 : Fri : None
 2025-08-02 : Sat : None
 2025-08-03 : Sun : None
 2025-08-04 : Mon : None
 2025-08-05 : Tue : None
 2025-08-06 : Wed : None
 2025-08-07 : Thu : None
 2025-08-08 : Fri : None
 2025-08-09 : Sat : None]


## AlwaysProvider

This is a trivial provider that always provides the given value

In [15]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
items = zip(days, map(AlwaysProvider('My value').get, days))
print_value_list(items)

[2025-07-31 : Thu : My value
 2025-08-01 : Fri : My value
 2025-08-02 : Sat : My value
 2025-08-03 : Sun : My value
 2025-08-04 : Mon : My value
 2025-08-05 : Tue : My value
 2025-08-06 : Wed : My value
 2025-08-07 : Thu : My value
 2025-08-08 : Fri : My value
 2025-08-09 : Sat : My value]


## ScheduledProvider

This provider provides its value according to the specified schedule. If not scheduled it provides `None`.

In [16]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
items = zip(days, map(ScheduledProvider('My value',
                                        AnySchedule({'Tuesdays': WeeklySchedule(TUESDAY),
                                                     'Thursdays': WeeklySchedule(THURSDAY)})).get, days))
print_value_list(items)

[2025-07-31 : Thu : My value
 2025-08-01 : Fri : None
 2025-08-02 : Sat : None
 2025-08-03 : Sun : None
 2025-08-04 : Mon : None
 2025-08-05 : Tue : My value
 2025-08-06 : Wed : None
 2025-08-07 : Thu : My value
 2025-08-08 : Fri : None
 2025-08-09 : Sat : None]


## AnyProvider

This provider takes a list of providers and provides the value from the first provider that provides a not `None` value. If all
the providers provide `None` then `None` will be provided.

In [17]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
items = zip(days, map(AnyProvider([ScheduledProvider('Value 1', UntilSchedule(START_DATE + timedelta(days=3))),
                                   ScheduledProvider('Value 2', UntilSchedule(START_DATE + timedelta(days=7))),
                                   ScheduledProvider('Value 3', UntilSchedule(START_DATE + timedelta(days=9)))]).get,
                      days))
print_value_list(items)

[2025-07-31 : Thu : Value 1
 2025-08-01 : Fri : Value 1
 2025-08-02 : Sat : Value 1
 2025-08-03 : Sun : Value 2
 2025-08-04 : Mon : Value 2
 2025-08-05 : Tue : Value 2
 2025-08-06 : Wed : Value 2
 2025-08-07 : Thu : Value 3
 2025-08-08 : Fri : Value 3
 2025-08-09 : Sat : None]


## AllProvider

This provider takes a dictionary of providers and provides a corresponding dictionary of the values provided by each provider

In [18]:
days = [START_DATE + timedelta(days=i) for i in range(10)]
items = zip(days, map(AllProvider({'Always': AlwaysProvider('My value'),
                                   'Scheduled': ScheduledProvider('My value',
                                                                  AnySchedule({'Tuesdays': WeeklySchedule(TUESDAY),
                                                                               'Thursdays': WeeklySchedule(THURSDAY)})),
                                   'Any': AnyProvider(
                                       [ScheduledProvider('Value 1', UntilSchedule(START_DATE + timedelta(days=3))),
                                        ScheduledProvider('Value 2', UntilSchedule(START_DATE + timedelta(days=7))),
                                        ScheduledProvider('Value 3',
                                                          UntilSchedule(START_DATE + timedelta(days=9)))])}).get, days))
print_value_list(items)

[2025-07-31 : Thu : {'Always': 'My value', 'Scheduled': 'My value', 'Any': 'Value 1'}
 2025-08-01 : Fri : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 1'}
 2025-08-02 : Sat : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 1'}
 2025-08-03 : Sun : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 2'}
 2025-08-04 : Mon : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 2'}
 2025-08-05 : Tue : {'Always': 'My value', 'Scheduled': 'My value', 'Any': 'Value 2'}
 2025-08-06 : Wed : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 2'}
 2025-08-07 : Thu : {'Always': 'My value', 'Scheduled': 'My value', 'Any': 'Value 3'}
 2025-08-08 : Fri : {'Always': 'My value', 'Scheduled': None, 'Any': 'Value 3'}
 2025-08-09 : Sat : {'Always': 'My value', 'Scheduled': None, 'Any': None}]
